# How to Use VTKOverlayWindow
scikit-surgeryvtk provides a simple PySide2/VTK widget called VTKOverlayWindow for doing simple augmented reality (AR) overlays, using calibrated parameters, like those obtained from OpenCV. The video image is rendered as a background layer, with VTK models as a foreground layer. This is not very photo-realistic, but most researchers are focussing on registration/alignment rather than true 3D perception.

In this page, we give a brief overview of the VTKOverlayWindow, a more detailed tutorial can be found in [SciKit-Surgery Augmented Reality Tutorial](https://scikit-surgerytutorial01.readthedocs.io/en/latest/)

The Jupyter notebook for this page can be found in the `docs/tutorials` folder of the [code repository](https://github.com/UCL/scikit-surgeryvtk/).

## Required modules
This tutorial requires numpy, scikit-surgeryvtk, scikit-surgeryutils and opencv-contrib-python.

## Quick implementation
One of the most common uses is to overlay some models on a camera feed, and a sample implementation is included in the scikit-surgeryutils module, which requires only a few lines of code.

**Note** - Running Qt Widgets from a Jupyter notebook can be a bit temperemental. Remember to close any Qt windows once you have finished with them, before moving onto a different section. If there are any issues, restarting the Jupyter kernel should be sufficient to fix.

The code can also be run as a standard .py script file.

In [3]:
import sksurgeryutils.common_overlay_apps as coa
from PySide2 import QtWidgets, QtCore

The below code will launch a Qt widget in a new window, where you should be able to interact with the loaded models (drag/rotate etc).

In [4]:
import os
print(os.getcwd())

E:\Code\Python\scikit-surgery\scikit-surgeryvtk\docs\tutorials


In [ ]:
app = QtCore.QCoreApplication.instance()
if app is None:
    app = QtWidgets.QApplication([])

camera_source = 0
wind = coa.OverlayOnVideoFeed(camera_source)
wind.add_vtk_models_from_dir('../../tests/data/models/Liver')
wind.start()

app.exec_()

## Detailed Implementation
The `OverlayOnVideoFeed` class has an `update` method which is called to read a new frame from the webcam and display it in the widget. By writing our own `update` method, we can add additional functionality.

In [3]:
import cv2
from PySide2 import QtWidgets, QtCore
import sksurgeryvtk.widgets.vtk_overlay_window as OW
import sksurgeryvtk.models.vtk_surface_model as SM
import sksurgeryvtk.models.vtk_surface_model_directory_loader as SMDL

Now we define a new class, using a `QTimer` to call the `update` method, to which we can add whatever processing is desired. In this instance it just flips the video once it has been received.

In [5]:
class OverlayApp():
    
    def __init__(self, video_source):
        self.vtk_overlay_window = OW.VTKOverlayWindow()
        self.video_source = cv2.VideoCapture(video_source)
        
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.update)

        update_frequency_ms = 100
        self.timer.start(update_frequency_ms)
        
        self.vtk_overlay_window.show()
        
    def update(self):
        ret, img = self.video_source.read()
        img = cv2.flip(img, 0)
        self.vtk_overlay_window.set_video_image(img)
        self.vtk_overlay_window._RenderWindow.Render()

In [7]:
app = QtCore.QCoreApplication.instance()
if app is None:
    app = QtWidgets.QApplication([])

camera_source = 0
overlay_app = OverlayApp(camera_source)

app.exec_()

0

Overlay models can be loaded from a directory:

In [6]:
app = QtCore.QCoreApplication.instance()
if app is None:
    app = QtWidgets.QApplication([])

camera_source = 0
overlay_app = OverlayApp(camera_source)

model_dir = '../../tests/data/models/Liver'
model_loader = SMDL.VTKSurfaceModelDirectoryLoader(model_dir)

overlay_app.vtk_overlay_window.add_vtk_models(model_loader.models)

app.exec_()

0

Or from individual files. It is also possible to set the colour and opacity individually for each model:

In [ ]:
app = QtCore.QCoreApplication.instance()
if app is None:
    app = QtWidgets.QApplication([])

camera_source = 0
overlay_app = OverlayApp(camera_source)

liver_file= '../../tests/data/models/Liver/liver.vtk'
portal_vein_file = '../../tests/data/models/Liver/portal_vein.vtk'

liver_model = SM.VTKSurfaceModel(liver_file, colour = [1.0, 0.0, 0.0], opacity = 0.5)
vein_model = SM.VTKSurfaceModel(portal_vein_file, colour = [0.0, 1.0, 0.0])


overlay_app.vtk_overlay_window.add_vtk_models([liver_model, vein_model])

app.exec_()

## Setting Camera Parameters

If you want to use a calibrated camera, it is assumed that the rendering is to be done in undistorted space. So, you would provide an undistorted image to the `set_video_image` method. This means that to get a correct AR overlay, you simply need to set the camera intrinsic parameters, and the camera extrinsics (pose, position and orientation).

The extrinsics are defined as being the camera to world matrix, i.e. to place the camera at the correct position in the world. If you are trying to set the pose to verify an OpenCV camera calibration, and are using the OpenCV extrinsic matrix from the calibration process, then this is in effect a transformation from model (chessboard) to camera, so you’d need to invert the OpenCV extrinsic matrix.

Once you have appropriate parameters, you can pass them to the overlay window:

```python
vtk_overlay_window = VTKOverlayWindow()
intrinsics = # get intrinsics somehow, 3x3 numpy array
vtk_overlay_window.set_camera_matrix(intrinsics) 
vtk_overlay_window.show()

while(True):

  undistorted_rgb_image = # get undistorted RGB image somehow.
  vtk_overlay_window.set_video_image(undistorted_rgb_image)

  camera_to_world = # compute camera to world transform
  vtk_overlay_window.set_camera_pose(camera_to_world)
 ```